
# Loading Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_theme(style="whitegrid")
import matplotlib.pyplot as plt
%matplotlib inline

import datetime
import os
import cv2

import tensorflow as tf
from tensorflow.keras.optimizers import Adam, RMSprop, SGD

from keras import regularizers
from keras.layers import Conv2D, Dense, BatchNormalization, Activation, Dropout, MaxPooling2D, Flatten
from keras.callbacks import ModelCheckpoint, CSVLogger, TensorBoard, EarlyStopping, ReduceLROnPlateau
from keras_preprocessing.image import ImageDataGenerator, load_img
from keras.utils.vis_utils import plot_model

from sklearn.metrics import classification_report, confusion_matrix

main_accent_colour = "#b366ff"
dim_colour="darkgrey"
main_palette = ["#FBE5C0", "#DD9A30", "#F88379", "#FF6FC2", "purple", "#D086F6", "#B0D2C2", "#4C5D70", "#6FA2CE", "#382D24", "#3ACF3A", "#7D7D00"]

# image capture 

In [ ]:
import cv2

cam = cv2.VideoCapture(0)

cv2.namedWindow("test")

img_counter = 0

while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
    cv2.imshow("test", frame)

    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed
        img_name = "opencv_frame_{}.png".format(img_counter)
        cv2.imwrite(img_name, frame)
        print("{} written!".format(img_name))
        img_counter += 1

cam.release()

cv2.destroyAllWindows()
print(img_name)

# Analyzing

In [ ]:
train_dir = 'C:/Users/KIIT/train/'
test_dir = 'C:/Users/KIIT/test/'

row = 48
col = 48
classes = len(os.listdir('C:/Users/KIIT/train/'))

In [ ]:
print("Train Set :")

train_count = []
for folder in os.listdir(train_dir) :
    print(folder, "folder contains\t\t", len(os.listdir(train_dir+folder)), "image")
    train_count.append(len(os.listdir(train_dir+folder)))
    
print()

test_count = []
print("Test Set :")
for folder in os.listdir(test_dir) :
    print(folder, "folder contains\t\t", len(os.listdir(test_dir+folder)), "images")
    test_count.append(len(os.listdir(test_dir+folder)))

In [ ]:
vals = ["disgust"]
palette = {c: dim_colour if c not in vals else main_accent_colour for c in os.listdir(train_dir)}

plt.figure(figsize=(8,4))

ax = sns.barplot(y=os.listdir(train_dir),
                 x=train_count,
                 palette=palette,
                 orientation="horizontal",
).set(title='Train Classes')

plt.show()

print()

ax = sns.barplot(y=os.listdir(test_dir),
                 x=test_count,
                 palette=palette,
                 orientation="horizontal",
).set(title='Test Classes')

plt.show()

print()

plt.figure(figsize=(25,25))

i = 1
for folder in os.listdir(train_dir):
    
    img = load_img((train_dir + folder +'/'+ os.listdir(train_dir + folder)[1]))
    plt.subplot(1,7,i)
    plt.imshow(img)
    plt.title(folder)
    plt.axis('off')
    i += 1
    
plt.show()

# Preprocessing

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   zoom_range=0.3,
                                   horizontal_flip=True)


training_set = train_datagen.flow_from_directory(train_dir,
                                                batch_size=64,
                                                target_size=(48,48),
                                                shuffle=True,
                                                color_mode='grayscale',
                                                class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(test_dir,
                                                batch_size=64,
                                                target_size=(48,48),
                                                shuffle=True,
                                                color_mode='grayscale',
                                                class_mode='categorical')

In [ ]:
training_set.class_indices

# Model

In [ ]:
def get_model(input_size, classes=7):
    
    model = tf.keras.models.Sequential()   

    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape =input_size))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2, 2))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(classes, activation='softmax'))

    model.compile(optimizer=Adam(lr=0.0001, decay=1e-6), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    return model

In [ ]:
fernet = get_model((row,col,1), classes)
fernet.summary()

In [ ]:
plot_model(fernet, show_layer_names=True)

# Checkpoint

In [ ]:
chk_path = 'ferNet.h5'
log_dir = "checkpoint/logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

checkpoint = ModelCheckpoint(filepath=chk_path,
                             save_best_only=True,
                             verbose=1,
                             mode='min',
                             moniter='val_accuracy')

earlystop = EarlyStopping(monitor='val_accuracy', 
                          min_delta=0, 
                          patience=3, 
                          verbose=1, 
                          restore_best_weights=True)
                        
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                              factor=0.2, 
                              patience=6, 
                              verbose=1, 
                              min_delta=0.0001)


tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
csv_logger = CSVLogger('training.log')

callbacks = [checkpoint, reduce_lr, csv_logger]

In [ ]:
steps_per_epoch = training_set.n // training_set.batch_size
validation_steps = test_set.n // test_set.batch_size

hist = fernet.fit(x=training_set,
                  validation_data=test_set,
                  epochs=48,
                  callbacks=callbacks,
                  steps_per_epoch=steps_per_epoch,
          
                  validation_steps=validation_steps)

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')

In [ ]:
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')

In [ ]:
train_loss, train_accu = fernet.evaluate(training_set)
test_loss, test_accu = fernet.evaluate(test_set)
print("final train accuracy = {:.2f} , validation accuracy = {:.2f}".format(train_accu*100, test_accu*100))

# Result

In [ ]:
y_pred = fernet.predict(training_set)
y_pred = np.argmax(y_pred, axis=1)
class_labels = test_set.class_indices
class_labels = {v:k for k,v in class_labels.items()}

cm_train = confusion_matrix(training_set.classes, y_pred)
print('Confusion Matrix')
print(cm_train)
print('Classification Report')
target_names = list(class_labels.values())
print(classification_report(training_set.classes, y_pred, target_names=target_names))

plt.figure(figsize=(8,8))
plt.imshow(cm_train, interpolation='nearest')
plt.colorbar()
tick_mark = np.arange(len(target_names))
_ = plt.xticks(tick_mark, target_names, rotation=90)
_ = plt.yticks(tick_mark, target_names)

In [ ]:
fernet.save("2130094")


In [ ]:
import tensorflow.keras.models

In [ ]:
models = tensorflow.keras.models.load_model('2130094')

In [ ]:
models

# Recommending Music And Movies


In [ ]:
mood_music = pd.read_csv("C:/Users/KIIT/data_moods.csv")
mood_music = mood_music[['name','artist','mood']]
mood_music

In [ ]:
image = cv2.imread(img_name,cv2.IMREAD_GRAYSCALE)

plt.imshow(image,cmap='gray')
plt.show()

In [ ]:
image = cv2.resize(image,(48,48))
img=np.array(image)
img=img.reshape(1,48,48,1)
predict_x=models.predict(img) 
result=np.argmax(predict_x,axis=1)
result[0]

In [ ]:
if(result[0]==0 or result[0]==1 or result[0]==2 ):
    #for angery,disgust,fear
    filter1=mood_music['mood']=='Calm'
    f1=mood_music.where(filter1)
    f1=f1.dropna()
    f2 =f1.sample(n=5)
    f2.reset_index(inplace=True)
    display(f2)
if(result[0]==3 or result[0]==4):
    #for happy, neutral
    filter1=mood_music['mood']=='Happy'
    f1=mood_music.where(filter1)
    f1=f1.dropna()
    f2 =f1.sample(n=5)
    f2.reset_index(inplace=True)
    display(f2)
if(result[0]==5):
       #for Sad
    filter1=mood_music['mood']=='Sad'
    f1=mood_music.where(filter1)
    f1=f1.dropna()
    f2 =f1.sample(n=5)
    f2.reset_index(inplace=True)
    display(f2)
if(result[0]==6):
     #for surprise
    filter1=mood_music['mood']=='Energetic'
    f1=mood_music.where(filter1)
    f1=f1.dropna()
    f2 =f1.sample(n=5)
    f2.reset_index(inplace=True)
    display(f2)


In [ ]:
movie=pd.read_csv("movies.csv")
mood_movie=movie[['title','genres']]
movie



In [ ]:
filter1=mood_movie["genres"]
j=0
if(result[0]==0 or result[0]==1 or result[0]==2 ):
    #for angery,disgust,fear
    filter1=mood_movie["genres"]
    for i in filter1:
        l1=list(i.split("|"))
        if"Comedy" in l1:
            print(mood_movie.iloc[j])
        j+=1
if(result[0]==3 or result[0]==4):
    #for happy, neutral
    filter1=mood_movie["genres"]
    for i in filter1:
        l1=list(i.split("|"))
        if "Adventure" and "Mystery" in l1:
            print(mood_movie.iloc[j])
        j+=1
if(result[0]==5):
       #for Sad
     filter1=mood_movie["genres"]
     for i in filter1:
        l1=list(i.split("|"))
        if "Action" and "Sci-Fi" in l1:
            print(mood_movie.iloc[j])
        j+=1
if(result[0]==6):
     #for surprise
     filter1=mood_movie["genres"]
     for i in filter1:
        l1=list(i.split("|"))
        if "Drama" and "Documentry" in l1:
            print(mood_movie.iloc[j])
        j+=1

# chatbot

In [ ]:
data={"intents": [
        
        {"tag": "success",
             "patterns": ["hey I completed my project", "I got selected", "I got promoted" ],
         "responses": ["Congrats!you deserved it","Party time!! so happy for you","So proud of you"],
         "context": [""]
        },
        {"tag": "disgusted",
             "patterns": ["he spits while eating", "she is creepy", "he is always making me uncomfortable" ],
         "responses": ["you can talk to them about this","you are too good for h","You don't have to be around that person"],
         "context": [""]
        },
        {"tag": "angry",
             "patterns": ["this is such a waste of time", "I'm so done now", "I am so angry about it" ],
         "responses": ["Calm down you'll be fine","It's okay I am always here","Hey champ don't waste yourself"],
         "context": [""]
        },
        {"tag": "travel",
             "patterns": ["I wish to travel", "I want to explore myself", "I want to find as new hobby" ],
         "responses": ["Sure dude that's awesome","good to see you like this","That's a rocking idea"],
         "context": [""]
        },
        {"tag": "greet",
             "patterns": ["Hi", "Hey", "How are you?" ],
         "responses": ["Hi good to see you","hey dude what's up?","hey I'm great how about you?"],
         "context": [""]
        },
        {"tag": "life",
             "patterns": ["what is life?", "when will I succeed?", "Why do I fail?" ],
         "responses": ["Good things take time","It's okay, it's just a bad phase","come on you will be good"],
         "context": [""]
        },
        {"tag": "music",
        "patterns": ["what's your favorite song?", "who's your favorite artist?", "do you like jazz?" ],
        "responses": ["I like all kinds of music", "I'm not really into music, sorry", "I love classical music"],
        "context": [""]
        },
        {"tag": "movies",
        "patterns": ["what's your favorite movie?", "have you seen the latest Marvel movie?", "what's the best action movie?" ],
        "responses": ["I'm a chatbot, so I don't really watch movies", "I haven't watched any good movies lately", "I love horror movies"],
        "context": [""]
        },
        {"tag": "books",
        "patterns": ["what's your favorite book?", "have you read the latest Stephen King novel?", "what's the best self-help book?" ],
        "responses": ["I don't read books, sorry", "I haven't read any good books lately", "I love mystery novels"],
        "context": [""]
        },
        {"tag": "food",
        "patterns": ["what's your favorite cuisine?", "have you tried Indian food?", "what's the best restaurant in town?" ],
        "responses": ["I'm a chatbot, so I don't really eat food", "I haven't tried any new restaurants lately", "I love Italian food"],
        "context": [""]
        },
        {"tag": "sports",
        "patterns": ["what's your favorite sport?", "have you watched the latest soccer game?", "what's the best sports team?" ],
        "responses": ["I'm a chatbot, so I don't really watch sports", "I haven't watched any good games lately", "I love basketball"],
        "context": [""]
        },
        {"tag": "technology",
        "patterns": ["what's your favorite tech gadget?", "have you tried the latest iPhone?", "what's the best laptop for gaming?" ],
        "responses": ["I'm a chatbot, so I don't really use technology", "I haven't tried any new gadgets lately", "I love Apple products"],
        "context": [""]
        },
        {"tag": "animals",
        "patterns": ["what's your favorite animal?", "have you seen a panda before?", "what's the best pet for kids?" ],
        "responses": ["I'm a chatbot, so I don't really have a favorite animal", "I haven't seen any interesting animals lately", "I love cats"],
        "context": [""]
        },
        {"tag": "politics",
        "patterns": ["what's your political view?", "have you been following the latest election?", "who's the best politician?" ],
        "responses": ["I don't have a political view, sorry", "I haven't been following politics lately", "I don't have an opinion on politicians"],
        "context": [""]
        },
        {"tag": "nature",
        "patterns": ["what's your favorite outdoor activity?", "what's the best place to hike?" ],
        "responses": ["Being in sunshine and out is one's best part of life", "I love hiking in the mountains"],
        "context": [""]
        },
        {
        "tag": "love",
        "patterns": ["I'm in love","How do I know if I'm in love?","I can't stop thinking about them"],
        "responses": ["Love is a beautiful feeling","It sounds like you have strong feelings for them","Love is all about taking a leap of faith"],
        "context": [""]
        },
        {
        "tag": "breakup",
        "patterns": ["I just went through a breakup","How do I get over my ex?","I miss them so much"],
        "responses": ["I'm sorry to hear that","Breakups are tough but you will get through this","Take some time for yourself"],
        "context": [""]
        },
        {
        "tag": "anxiety",
        "patterns": ["I'm feeling anxious","What do I do when I have an anxiety attack?","How do I cope with anxiety?"],
        "responses": ["Take some deep breaths","Try to focus on the present moment","It's okay to ask for help"],
        "context": [""]
        },
        {
        "tag": "depression",
        "patterns": ["I'm feeling depressed","How do I deal with depression?","I don't enjoy anything anymore"],
        "responses": ["It's okay to not be okay","Depression is a common issue, but it can be treated","Have you considered seeking professional help?"],
        "context": [""]
        },
        {
        "tag": "motivation",
        "patterns": ["I need motivation","How do I stay motivated?","I'm feeling demotivated"],
        "responses": ["Think about why you started in the first place","Take small steps towards your goal","You got this!"],
        "context": [""]
        },
        {
        "tag": "stress",
        "patterns": ["I'm feeling stressed","How do I manage stress?","I can't handle the pressure"],
        "responses": ["Take a break and relax","Try some stress-relieving activities like yoga or meditation","Don't hesitate to ask for help"],
        "context": [""]
        },
        {
        "tag": "self-care",
        "patterns": ["I need to practice self-care","How do I take care of myself?","I'm neglecting my needs"],
        "responses": ["Self-care is important, make sure to prioritize it","Take some time for yourself and do something you enjoy","Remember that you deserve to take care of yourself"],
        "context": [""]
        },
        {"tag": "relationships",
        "patterns": ["How do I improve my relationship?","I'm having issues with my partner","What makes a good relationship?"],
        "responses": ["Communication is key in any relationship","Try to understand each other's perspectives","Remember to respect each other"],
        "context": [""]
        }   
        ]
}


# IMPORTING LIBRARY

In [ ]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

# PREPROCESSING 

In [ ]:
training_sentences = []
training_labels = []
labels = []
responses = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])   
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)


In [ ]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [ ]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"


tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [ ]:
model1 = Sequential()
model1.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model1.add(GlobalAveragePooling1D())
model1.add(Dense(16, activation='relu'))
model1.add(Dense(16, activation='relu'))
model1.add(Dense(num_classes, activation='softmax'))

model1.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model1.summary()

In [ ]:
epochs = 200
history = model1.fit(padded_sequences, np.array(training_labels), epochs=epochs)

In [ ]:
train_loss1, train_accu1 = model1.evaluate(training_set)
test_loss1, test_accu1 = model1.evaluate(test_set)
print("final train accuracy = {:.2f} , validation accuracy = {:.2f}".format(train_accu1*100, test_accu1*100))

In [ ]:
# to save the trained model
model1.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
print(data)

In [ ]:
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')

In [ ]:
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')